In [ ]:
%pip install pandas openpyxl deep-translator

In [ ]:
import pandas as pd
from deep_translator import GoogleTranslator

df = pd.read_excel("/home/roman/Desktop/Роман Грибов - dialog_talk_agent.xlsx")

df_valid = df.dropna(subset=["Text Response"]).copy()
print(f"Строк с ответами: {len(df_valid)}")

translator = GoogleTranslator(source='en', target='ru')

def safe_translate(text):
    return translator.translate(str(text).strip())

print("Перевод Context...")
df_valid["Context_ru"] = df_valid["Context"].astype(str).apply(safe_translate)

print("Перевод Text Response...")
df_valid["Text Response_ru"] = df_valid["Text Response"].astype(str).apply(safe_translate)

output_file = "dialog_talk_agent_ru_clean.xlsx"
df_valid.to_excel(output_file, index=False)

print(f"\nСохранено: {output_file}")

In [2]:
import pandas as pd
import spacy

df = pd.read_excel("dialog_talk_agent_ru_clean.xlsx")

nlp = spacy.load("ru_core_news_sm")

def lemmatize_ru(text):
    if not isinstance(text, str) or not text.strip():
        return ""
    doc = nlp(text.lower())
    lemmas = []
    for token in doc:
        if token.is_punct or token.is_space:
            continue
        lemmas.append(token.lemma_)
    return " ".join(lemmas)

print("Лемматизация Context_ru...")
df["Lemmas_ru"] = df["Context_ru"].apply(lemmatize_ru)

df.to_excel("dialog_talk_agent_ru_lemmatized.xlsx", index=False)
print("✅ Лемматизация завершена!")

Лемматизация Context_ru...
✅ Лемматизация завершена!


In [53]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy

df = pd.read_excel("dialog_talk_agent_ru_lemmatized.xlsx")

vectorizer = TfidfVectorizer(
    lowercase=False,   
    stop_words=None   
)
tfidf_matrix = vectorizer.fit_transform(df["Lemmas_ru"])

nlp = spacy.load("ru_core_news_sm")

def lemmatize_query(text):
    if not isinstance(text, str) or not text.strip():
        return ""
    doc = nlp(text.lower())
    lemmas = [token.lemma_ for token in doc if not token.is_punct and not token.is_space]
    return " ".join(lemmas)

def get_response(user_input: str) -> str:
    query_lem = lemmatize_query(user_input)
    
    query_vec = vectorizer.transform([query_lem])
    
    sims = cosine_similarity(query_vec, tfidf_matrix).flatten()
    
    best_idx = sims.argmax()
    best_score = sims[best_idx]
    
    if best_score < 0.1:
        return "Что чёрт возьми ты несёшь?"
    
    return df.iloc[best_idx]["Text Response_ru"], best_score, df.iloc[best_idx]["Context_ru"]

print(get_response("Ты такой чудак"))
print(get_response("грустно"))
print(get_response("Где твой дом?"))

('Ваше восприятие. Моя реальность.', np.float64(0.7929524434582053), 'ты чудак')
('О, нет. В чем дело?', np.float64(0.8240422779108789), 'Мне грустно')
('Виртуальный мир — моя игровая площадка. Я всегда на расстоянии нескольких кликов мыши.', np.float64(0.7856314550015997), 'твой дом')


Что ж, решил перевести тот английский датасет из гугл класса, работает с ооочень переменным успехом, тут скорее трудности перевода